In [1]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('./src/')

In [2]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import folium
from package import storage
from package import tracer
from package.tracer.folium import add_tracer_list_to_folium_map

In [3]:
stops_df = storage.read_df('./data/cleaned/stops.csv')
stops_gdf = gpd.GeoDataFrame(stops_df, geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat))

In [4]:
# earliest_arrival_times = pd.read_csv('./data/raptor_results/arrival_times.csv')
earliest_arrival_times = storage.read_df('/home/moritz/dev/uni/mcr-py/src/testdata/output/arrival_times.csv')
earliest_arrival_times.sort_values(by=['arrival_time'])
earliest_arrival_times["stop_id"] = earliest_arrival_times["stop_id"].astype(str)
earliest_arrival_times = earliest_arrival_times.set_index('stop_id')
earliest_arrival_times["is_reached"] = earliest_arrival_times["arrival_time"] != "--:--:--"
earliest_arrival_times

,arrival_time,is_reached
stop_id,,
305,15:24:00,True
833,15:35:19,True
771,15:39:03,True
34,15:13:00,True
326,15:26:00,True
...,...,...
7394,15:21:39,True
885,15:13:55,True
752,15:20:00,True


In [5]:
earliest_arrival_times.loc["835", "arrival_time"]

'15:33:27'

In [6]:
tracer_map = storage.read_any_dict("./data/raptor_results/tracer_map.pkl")["tracer_map"]
current = tracer_map["835"]
current

[Start at 818 at 15:00:00,
 Trip 424-16-001-343.1.12:150700-20-257_911E53F4-F87D-4A5E-8069-B02D012A72BB from 818@15:08:00 to 317@15:09:00,
 Trip 1462-13-001-596.1.12:150300-26-257_4163D12A-EFFE-4C91-911B-B02D012A8323 from 317@15:20:00 to 251@15:31:00,
 Walk from 251 to 835 in 00:02:27]

In [7]:
from package.output import enrich_raptor_trace_results
tracer_map =  enrich_raptor_trace_results("./data/raptor_results/", "./data/cologne_gtfs.zip")
current = tracer_map["835"]

In [8]:
sample_stop = stops_gdf.sample(1)
m = folium.Map(location=[sample_stop.stop_lat, sample_stop.stop_lon], zoom_start=12)
for i, row in stops_gdf.iterrows():
    eat = earliest_arrival_times.loc[row.stop_id]
    is_reached = eat["is_reached"]
    color = "green" if is_reached else "red"
    folium.CircleMarker(
        location=[row.stop_lat, row.stop_lon],
        radius=1,
        color=color,
        popup=f"{row.stop_name} ({row.stop_id}) @ {eat['arrival_time']}",
    ).add_to(m)
m
add_tracer_list_to_folium_map(current, m, stops_df)
m

/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:69: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]
